In [1]:
import numpy as np
from MainFunctions import *
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
from skimage import io, filters
import cv2

In [2]:
#original 512 x 512
i_width = 128
i_height = 128
input_shape = i_width*i_height
print(input_shape)

16384


In [3]:
x_trn = []
x_tst = []

AddImageToArray(x_trn ,'DICOM/PA2/ST1/SE4', (i_width, i_height))
AddImageToArray(x_tst ,'DICOM/PA5/ST1/SE4', (i_width, i_height))

x_trn = np.array(x_trn, np.float32)
x_tst = np.array(x_tst, np.float32)
print(x_trn.shape)

(269, 128, 128)


In [4]:
x_trn = segmentation(x_trn, 21,21)
x_tst = segmentation(x_tst, 25,16)

In [5]:
from sklearn.cluster import AffinityPropagation
clusterisation_method = AffinityPropagation(random_state=4114)
labels = clusterisation_method.fit_predict(np.reshape(x_trn, (-1, input_shape)))
output_shape = labels.max()+1


c:\Users\ivan_\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [6]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

y_categorical = to_categorical(labels, output_shape)

In [7]:
class build_LayerClassificator(nn.Module):
    def __init__(self, inp_shape, out_shape):
        super().__init__()
        """Classificator"""
        self.flatten = nn.Flatten()
        self.fully_connected1 = nn.Linear(int(inp_shape), out_shape)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        x = self.flatten(inputs)
        x = self.fully_connected1(x)
        x = self.softmax(x)
        return x

In [18]:
#use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = build_LayerClassificator(input_shape, output_shape).to(device)

In [23]:
def train(epochs, model, criterion, optimizer, train_loader, eps):
    for epoch in range(epochs):
        loss = 0
        for batch_features, targets in train_loader:
            batch_features = batch_features.view(-1, 1, i_height, i_width).to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            
            # compute reconstructions
            outputs = model(batch_features)
            # compute training reconstruction loss
            train_loss = criterion(outputs, targets)
            
            # compute accumulated gradients
            train_loss.backward()
            
            # perform parameter update based on current gradients
            optimizer.step()
            
            # add the mini-batch training loss to epoch loss
            loss += train_loss.item()
        
        # compute the epoch training loss
        loss = loss / len(train_loader)
        
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
        if loss < eps:
            break
    
    return model

In [24]:
# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

tensor_x = torch.Tensor(x_trn) # transform to torch tensor
tensor_y = torch.Tensor(y_categorical)

my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

train_loader = DataLoader(
    my_dataset, batch_size=64, shuffle=True
)

In [25]:
model = train(1000, model, criterion, optimizer, train_loader, 0.006)

epoch : 1/1000, loss = 0.023546
epoch : 2/1000, loss = 0.012068
epoch : 3/1000, loss = 0.009115
epoch : 4/1000, loss = 0.006652
epoch : 5/1000, loss = 0.006312
epoch : 6/1000, loss = 0.005843


In [26]:
with torch.no_grad():
    batch_features = torch.asarray(x_trn)
    batch_features = batch_features.view(-1, 1, i_height, i_width).to(device)
    pred = np.array(model(batch_features ))

In [27]:
print(labels-np.argmax(pred,1))

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
